<a href="https://colab.research.google.com/github/freecopyA4/-2/blob/main/pythonstudy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import pandas as pd
import numpy as np
from pandas import Series, DataFrame



db = json.load(open('/content/drive/MyDrive/1.2/ch07/foods-2011-10-03.json'))
# len(db)

# db[0].keys()
# db[0]['nutrients'][0]

# nutrients = DataFrame(db[0]['nutrients'])
# nutrients[:7]

info_keys = ['description', 'group', 'id', 'manufacturer']
info = DataFrame(db, columns=info_keys)
# info[:5]
# info.info()

# pd.value_counts(info.group)[:10]

nutrients = []

for rec in db:
  fnuts = DataFrame(rec['nutrients'])
  fnuts['id'] = rec['id']
  nutrients.append(fnuts)

nutrients = pd.concat(nutrients, ignore_index=True)

# nutrients

nutrients.duplicated().sum()
nutrients = nutrients.drop_duplicates()

col_mapping = {'description' : 'food',
               'group' : 'fgroup' }

info = info.rename(columns=col_mapping, copy=False)
# info.info()

col_mapping = {'description' : 'nutrient',
               'group' : 'nutgroup' }

nutrients = nutrients.rename(columns=col_mapping, copy=False)
# nutrients

ndata = pd.merge(nutrients, info, on='id', how='outer')
# ndata.info()
# ndata.loc[3000]

# result = ndata.groupby(['nutrient', 'fgroup'])['value'].quantile(0.5)
# result['Zinc, Zn'].sort_values().plot(kind='barh')

by_nutrient = ndata.groupby(['nutgroup', 'nutrient'])

get_maximum = lambda x: x.xs(x.value.idxmax())
get_minimum = lambda x: x.xs(x.value.idxmin())

max_foods = by_nutrient.apply(get_maximum)[['value', 'food']]
max_foods.food = max_foods.food.str[:50]

max_foods.loc['Amino Acids']['food']

In [ ]:
! pip install basemap

In [ ]:
import json
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
import matplotlib.pyplot as plt

data = pd.read_csv('/content/drive/MyDrive/Haiti.csv')
# data.info()
# data[['INCIDENT DATE', 'LATITUDE', 'LONGITUDE']][:10]
# data['CATEGORY'][:6]
# data.describe()
data = data[(data.LATITUDE > 18) & (data.LATITUDE < 20) & 
            (data.LONGITUDE > -75) & (data.LONGITUDE < -70)
            & data.CATEGORY.notnull()]

# 카테고리별 분석 및 시각화

def to_cat_list(catstr):
  stripped = (x.strip() for x in catstr.split(','))
  return [x for x in stripped if x]

def get_all_categories(cat_series):
  cat_sets = (set(to_cat_list(x)) for x in cat_series)
  return sorted(set.union(*cat_sets))

def get_english(cat):
  code, names = cat.split('.')
  if '|' in names:
    names = names.split('|')[1]
  return code, names.split()

# get_english('2. Urgences logistiques | Vital Lines')

all_cats = get_all_categories(data.CATEGORY)

# 제너레이터 표현

english_mapping = dict(get_english(x) for x in all_cats)
# english_mapping['2a']
# english_mapping['6c']

def get_code(seq):
  return [x.split('.')[0] for x in seq if x]

all_codes = get_code(all_cats)
code_index = pd.Index(np.unique(all_codes))
dummy_frame = DataFrame(np.zeros((len(data), len(code_index))),
                        index=data.index, columns=code_index)

# dummy_frame.iloc[:, :6].info()

for row, cat in zip(data.index, data.CATEGORY):
  codes = get_code(to_cat_list(cat))
  dummy_frame.loc[row, codes] = 1

data = data.join(dummy_frame.add_prefix('category_'))

# data.iloc[:, 10:15].info()

# 도표그리기

from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt

def basic_haiti_map(ax=None, lllat=17.25, urlat=20.25,
                    lllon=-75, urlon=-71):
  m = Basemap(ax=ax, projection='stere',
              lon_0=(urlon + lllon) / 2,
              lat_0=(urlat + lllat) / 2,
              llcrnrlat=lllat, urcrnrlat=urlat,
              llcrnrlon=lllon, urcrnrlon=urlon,
              resolution='i')
  m.drawcoastlines()
  m.drawstates()
  m.drawcountries()
  return m

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 10))
fig.subplots_adjust(hspace=0.05, wspace=0.05)

to_plot = ['2a', '1', '3c', '7a']

lllat=17.25; urlat=20.25; lllon=-75; urlon=-71

for code, ax in zip(to_plot, axes.flat):
  m = basic_haiti_map(ax, lllat=lllat, urlat=urlat,
                      lllon=lllon, urlon=urlon)
  cat_data = data[data['category_%s' % code] == 1]
  x, y = m(cat_data.LONGITUDE, cat_data.LATITUDE.values)

  m.plot(x, y, 'k.' , alpha=0.5)
  ax.set_title('%s: %s' % (code, english_mapping[code]))
  shapefile_path = '/content/drive/MyDrive/1.2/ch08/PortAuPrince_Roads'
  m.readshapefile(shapefile_path, 'roads')
plt.show



